### Making the necessary imports

In [1]:
import cv2
import numpy as np
import pandas as pd

from collections import Counter

### Function to display an image

In [2]:
def display(img) :
    screen_res = 1280, 720
    scale_width = screen_res[0] / img.shape[1]
    scale_height = screen_res[1] / img.shape[0]
    scale = min(scale_width, scale_height)
    window_width = int(img.shape[1] * scale) 
    window_height = int(img.shape[0] * scale)

    cv2.namedWindow('dst_rt', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('dst_rt', window_width, window_height)

    cv2.imshow('dst_rt', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## Best settings found for square detection so far

In [3]:
image = cv2.imread("sample_input.jpeg")
gray_image = cv2.cvtColor( image , cv2.COLOR_BGR2GRAY )
gray_image_blurred = cv2.GaussianBlur(gray_image,(13,13),0)

thresh = cv2.adaptiveThreshold(gray_image_blurred,255,1,1,11,2)

display(thresh)

#invert
#bw_image = 255 - bw_image

## Finding Contours

In [4]:
im2, contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [5]:
print ("Total no. of contours found in image are " , str(len(contours)))

Total no. of contours found in image are  370


### Finding The Largest Contour

In [6]:
biggest = None
max_area = 0
idi = None
idid,c = -1,0

for i in contours:
        area = cv2.contourArea(i) # Area of the contour
        if area > 100:
                peri = cv2.arcLength(i,True)  # Perimeter of the contour
                approx = cv2.approxPolyDP(i,0.02*peri,True) #Dimensions of the countour
                
                # (Top-Left X , Top-Left Y, Width, Height)
                (x, y, w, h) = cv2.boundingRect(approx) 
                
                #ratio to check whether the rectangle is a square or not
                #If it's a square , then ratio should be between 0.95 and 1.05
                ratio = w / float(h)
                
                if area > max_area and len(approx)==4 and ratio >= 0.90 and ratio <= 1.10 :
                        biggest = approx
                        max_area = area #Store the area of the largest contour
                        idi = i
                        idid = c #Stores the id of the largest contour
                        
                        
        c += 1                

### Plotting the largest contour

In [9]:
image2 = image.copy()
(x,y,w,h) = cv2.boundingRect(biggest)
print (cv2.boundingRect(biggest))
cv2.rectangle(image2,(x,y),(x+w-1,y+h-1),(0,0,255),5)

w_small = w / 9.0
h_small = h / 9.0

squares = []

for i in range(9) :
    row_squares = []
    for j in range(9) :
        #Vertical Lines
        x_first = x + int(round(i*w_small,3)) 
        y_first = y + int(round(j*h_small,3)) 

        x_end = x + int(round((i+1)*w_small,3)) 
        y_end = y + int(round((j+1)*h_small,3))
        
        #if (i % 3 == 0 and j % 3 == 0) :
        #cv2.rectangle(image2,(x_first,y_first),(x_end,y_end),(0,0,255),5)
        
        row_squares.append((x_first,y_first,x_end,y_end))
        
    squares.append(row_squares)
    
    
    

#cv2.drawContours(image2, contours, idid, (0,0,255), 5)
display(image2)

(29, 209, 662, 662)


## Model Part

In [10]:
train_data = pd.read_csv("train.csv", encoding = "UTF-8")

train_features = np.array(train_data)[:,1:]
labels = np.array(train_data)[:,:1]

train_features = train_features * ( 1.0 / 255.0 )

train_features[train_features > 0.0] = 1.0

In [11]:
#Contours_map stores the number of all-zeroes reigons in each digit
#contours_map[i] = Number of all-zeroes reigons in the digit i
contours_map = [2,1,1,1,2,1,2,1,3,2]

### K-Nearest Neighbours Model Class

In [12]:
class KNN() :
    
    def __init__(self,number_of_neighbours = 1) :
        #Storing the number of neighbours or K
        self.number_of_neighbours = number_of_neighbours
    
    #Training the Model
    def train(self,x_train,y_train) :
        self.x_train = x_train 
        self.y_train = y_train
    
    #Compute The Distances array
    #Distances array Contains the difference of this image from all other images in the training set
    def computeDistances(self,test) :
        
        #Here We'll use the L2 Distance or the Euclidean Distance
        
        #First We'll subtract the matrices
        difference = self.x_train - test
        
        #Then We'll square the difference matrix
        squared_difference = np.square( difference )
        
        #Then We'll add the squared_difference matrix rowwise
        summed_square_difference = np.sum( squared_difference , axis = 1 )
        
        #Finally We'll take the square Root
        differences = np.sqrt(summed_square_difference)
        
        return differences
    
    
    def predict(self,x_test,cc) :
        
        num_test = x_test.shape[0]
        
        final_output = np.zeros(num_test)
        
        for i in range(num_test) :
            
            #Getting the differences array
            differences = self.computeDistances( x_test[i] )
            
            #Sorting The differences array in reverse order
            sorted_differences = np.argsort( differences )
            
            #Slicing the first K(number_of_neighbours) values from the differences array
            first_k_differences = sorted_differences[:self.number_of_neighbours]
            
            #Checking whether the selected digit contains correct no. of all-zeroes reigons
            #according to our contour map
            c = 0 
            labels = []
            for j in sorted_differences :
                val = int(self.y_train[j]) 
                if (contours_map[val] != cc) :
                    continue 
                c += 1
                labels.append(val)
                if (c == self.number_of_neighbours) :
                    break
            
            #Getting the output Labels of each element in differences array
            #print (labels)
            
           
            
            #Dictionary Storing the count of each digit or label
            count_dict = Counter(labels)
            
            #Now We Need to find the most repated label out of these K-labels 
            #If there are multiple labels with the same count , then
            #We select the one which appears first in the labels list
            
            #Maxi stores the count of label that occurs maximum times
            maxi = 0 
            #The Selected Label
            selected_digit = None
            
            for j in range(self.number_of_neighbours) :
                
                label = labels[j]
                
                if count_dict[label] > maxi :
                    maxi = count_dict[label]
                    selected_digit = label
            
            final_output[i] = selected_digit
            
        return list(final_output)        

In [13]:
#Creating our KNN model where value of k (number of neighbours) = 10
final_model = KNN(10) 
final_model.train( train_features, labels )

### Extra Check functions for certain group of digits

In [14]:
def get_contours(img) :
    im2, contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return int(len(contours))

In [15]:
# This function classifies whether given digit is 1 or 7

def check17(img) :
    #print ("in " , img.shape)
    #check for 4
    low , high, col = 30 , -1 , None 
    for i in range(28) :
        for j in range(28) :
            if img[i,j]:
                low = min(low,i)
                high = max(high,i)
    
    for j in range(28) :
        if img[low,j] == 1 and img[high,j] == 1 :
            check = True
            for i in range(low,high+1) :
                if (img[i,j] != 1) :
                    check = False 
            if check :
                return 1
    return 7    

In [16]:
# This function classifies whether given digit is 4 or 9 or 6

def check496(img) :
    #print ("in " , img.shape)
    #check for 4
    low , high, col = 30 , -1 , None 
    for i in range(28) :
        for j in range(28) :
            if img[i,j]:
                low = min(low,i)
                high = max(high,i)
    #print (low,high)
    for j in range(28) :
        if img[low,j] == 1 and img[high,j] == 1 :
            #print ("in",j)
            check = True
            for i in range(low,high+1) :
                if (img[i,j] != 1) :
                    check = False 
            if check :
                return 4
    change = 0 
    for j in range(27) :
        if img[19,j] != img[19,j+1] and img[19,j+1] == 1 :
            change += 1
    if (change == 2) :
        return 6
    return 9   

### Actually Classifying the digits

In [17]:
conto = [[0] * 9]*9

In [18]:
from math import ceil
sudoku = []

small = None

ratios = int(w_small / 7.0)


for i in range(9) :
    row = []
    for j in range(9) :
        
        x_start,y_start,x_end,y_end = squares[i][j]
        
        block2 = thresh[y_start+ratios:y_end-(ratios-1),x_start+ratios:x_end-(ratios-1)]
        block2[block2 == 255] = 1.0
        block2[block2 == 0] = 0.0
        
        block2 = cv2.resize( block2 , (28,28))
        block_temp = block2
       
        
        for k in range(28) :
            for l in range(28) :
                if k <= 2 or l <= 2 or k >= 25 or l >= 25 :
                    block2[k,l] = 0
        
        temp = np.sum(np.sum(block2))
        if (temp == 0) :
            row.append(0) 
            continue 
        
        conto[i][j] = get_contours(block2)
        
        block2 = block2.flatten()
        
        
        temp_l = [block2]
        temp_l = np.array(temp_l)
        
        
        out = final_model.predict(temp_l,conto[i][j])
        #out = [0]
        val = int(out[0])
        if (val == 7 or val == 1) :
            val = check17(block_temp)
        elif (val == 9 or val == 4 or val == 6) :
            val = check496(block_temp)
    
       
        row.append(val)
        
    sudoku.append(row)
    print ("Done Row " + str(i+1))
    
    


Done Row 1
Done Row 2
Done Row 3
Done Row 4
Done Row 5
Done Row 6
Done Row 7
Done Row 8
Done Row 9


In [19]:
"""
#Debug script to print a particular sudoku subgrid
x_start,y_start,x_end,y_end = squares[1][5]

block_temp = thresh[y_start:y_end,x_start:x_end]

print (block_temp.shape)

block2 = thresh[y_start+ratios:y_end-(ratios-1),x_start+ratios:x_end-(ratios-1)]
block2[block2 == 255] = 1.0
block2[block2 == 0] = 0.0
        
block2 = cv2.resize( block2 , (28,28))
block_temp = block2
       
        
for k in range(28) :
    for l in range(28) :
        if k <= 2 or l <= 2 or k >= 25 or l >= 25 :
            block2[k,l] = 0
print (block2)
print (ratios)
"""

'\n#Debug script to print a particular sudoku subgrid\nx_start,y_start,x_end,y_end = squares[1][5]\n\nblock_temp = thresh[y_start:y_end,x_start:x_end]\n\nprint (block_temp.shape)\n\nblock2 = thresh[y_start+ratios:y_end-(ratios-1),x_start+ratios:x_end-(ratios-1)]\nblock2[block2 == 255] = 1.0\nblock2[block2 == 0] = 0.0\n        \nblock2 = cv2.resize( block2 , (28,28))\nblock_temp = block2\n       \n        \nfor k in range(28) :\n    for l in range(28) :\n        if k <= 2 or l <= 2 or k >= 25 or l >= 25 :\n            block2[k,l] = 0\nprint (block2)\nprint (ratios)\n'

### Printing the classified sudoku image

In [20]:
sudoku = np.array(sudoku).T
grid = sudoku.tolist()
grid2 = sudoku.tolist() 
print (sudoku)

[[0 3 1 6 0 7 0 0 0]
 [6 0 0 8 0 0 2 5 7]
 [8 0 0 0 9 0 0 0 3]
 [4 0 0 0 0 0 8 3 2]
 [0 1 0 0 0 9 0 0 0]
 [7 0 3 2 4 0 0 0 0]
 [0 0 2 4 0 1 0 7 8]
 [0 8 5 0 0 0 0 0 9]
 [3 0 4 0 0 0 0 6 1]]


### Sudoku Solver using BackTracking

In [21]:
def get_unassigned_loc() :
    global grid
    for i in range(9) :
        for j in range(9) :
            if (grid[i][j] == 0) :
                return i,j
    return -1,-1

In [22]:
def check(row,col) :
    #Check Row
    global grid
    freq = [0 for i in range(10)]
    for j in range(9) :
        if (grid[row][j] != 0) :
            freq[grid[row][j]] += 1
            if (freq[grid[row][j]] == 2) :
                return False
    
    #Check Col
    freq = [0 for i in range(10)]
    for j in range(9) :
        if (grid[j][col] != 0) :
            freq[grid[j][col]] += 1
            if (freq[grid[j][col]] == 2) :
                return False
    
    #Check Grid
    grow , gcol = int(row/3), int(col/3)
    freq = [0 for i in range(10)]
    for i in range(grow*3 , grow*3+3) :
        for j in range(gcol*3 , gcol*3+3) :
            if (grid[i][j] != 0) :
                freq[grid[i][j]] += 1
                if (freq[grid[i][j]] == 2) :
                    return False
                
    return True        

In [23]:
def solve() :
    global grid
    row,col = get_unassigned_loc()
    if (row+col==-2) :
        return True
    
    for num in range(1,10) :
        grid[row][col] = num 
        if (check(row,col) == False) :
            grid[row][col] = 0
            continue 
        
        if (solve()) :
            return True
        grid[row][col] = 0 
    return False   

In [24]:
%%time
solve()

CPU times: user 4.33 ms, sys: 186 µs, total: 4.52 ms
Wall time: 4.48 ms


True

### Printing the solved sudoku image

In [25]:
grid

[[5, 3, 1, 6, 2, 7, 9, 8, 4],
 [6, 4, 9, 8, 1, 3, 2, 5, 7],
 [8, 2, 7, 5, 9, 4, 6, 1, 3],
 [4, 9, 6, 1, 7, 5, 8, 3, 2],
 [2, 1, 8, 3, 6, 9, 7, 4, 5],
 [7, 5, 3, 2, 4, 8, 1, 9, 6],
 [9, 6, 2, 4, 5, 1, 3, 7, 8],
 [1, 8, 5, 7, 3, 6, 4, 2, 9],
 [3, 7, 4, 9, 8, 2, 5, 6, 1]]

### Printing the solved sudoku on the actual image

In [27]:
w = int(w_small)
h = int(h_small)

image5 = image.copy()

padding = int(1.5*ratios)

font = (2.0 * w_small) / 73 

for i in range(9) :
    for j in range(9) :
        if (grid2[j][i] == 0) :
            x_first = x + int(round(i*w_small,3)) 
            y_first = y + int(round(j*h_small,3)) 
            
            cv2.putText(image5, str(grid[j][i]), (x_first+padding, y_first+int(h_small)-padding), cv2.FONT_HERSHEY_SIMPLEX, font, (0, 0, 255),5)
            
            
display(image5)

cv2.imwrite('sample_output.png',image5)

True

### Successfully created the Sudoku-Solver